![diagram](Diagram.PNG)

## Plugin definition in common

### Manifest

In [1]:
# Manifest example

manifest = {
  'type': 'object',
  'title': 'RDP exploiter',
  'description': 'Exploits RDP.',
  'properties': {
    'safe': {
      'type': 'boolean',
      'title': 'is the exploiter safe to run'
    },
    'enabled': {
      'type': 'boolean',
      'title': 'is the exploiter enabled'
    },
    'link': {
      'type': 'string',
      'title': 'link to docs'
    },
    'plugin_options': {
      'type': 'object',
      'title': 'RDP exploiter options',
      'required': [
        'timeout',
        'rdp_versions'
      ],
      'properties': {
        'timeout': {
          'type': 'number',
          'title': 'login attempt timeout',
          'description': 'how long to wait for response to login attempt in seconds',
          'exclusiveMinimum': 0
        },
        'rdp_versions': {
          'type': 'array',
          'uniqueItems': true,
          'title': 'rpd versions',
          'description': 'rdp versions',
          'items': {
            // Devs can define references in manifest, but not if we use YAML
            'type': 'string',
            'title': 'RDPVersion',
            'anyOf': [
              {
                'type': 'string',
                'enum': ['1.1.+'],
                'title': '1.1.+'
              },
              {
                'type': 'string',
                'enum': ['2.+'],
                'title': '2.+'
              },
              {
                'type': 'string',
                'enum': ['3+'],
                'title': '3+'
              }
            ]
          }
        }
      }
  },
}

### Config

In [2]:
# Config example

config = {
    "plugin_name": "rdp_exploiter",
    "enabled": True,
    "safe": True,
    "link": "www.pork-face.com",
    # Don't validate these for now, as we only need the hard-coded smb-timeout
    "plugin_options":{
        "timeout": 10,
        "rdp_versions": ["2.+", "3+"]
    }
}

### Codebase

In [3]:
# Plugin codebase example (notice how it uses the options)
from time import sleep
from common.plugin_registry import register_plugin

def _scan_rdp_version(target):
    pass

def _attempt_brute_force(target, username, password):
    pass

def main(options):
    if not _scan_rdp_version(options['target']['ip']) in options["rdp_versions"]:
        raise RuntimeError(f"Target {options['target']} is running unsupported RDP version")
    for username, password in options['credentials']:
        if _attempt_brute_force(target, username, password):
            # Send exploitation event
            return
        sleep(options["scan_timeout"])

# IDK how we're going to register it, it's just a mock
register_plugin(manifest=manifest, config=config, entrypoint=main)

## Common actions

## Backend

### Loading the plugin

1. Load the plugin (make sure no other plugin with the same name already exists, validate the configuration against manifest)
2. Store configuration in the repository

### Resetting to default

1. Overwrite the configuration stored with the one in plugin definition

### Fetching agent configuration

1. Query an endpoint `/api/configuration/<plugin_name>`

### Setting configuration

1. Validate the configuration against manifest.
2. Store the configuration in repository. It's probably best to have a repository per config, file name would correspond to the name of the plugin.

## UI

### Loading configuration page
1. We need to create/mofify the schema based on manifests

### Exporting configuration

No changes

### Importing configuration

Standard procedure, but when it comes to plugins it would be best to import as much as we can. So if we have a config and have imported a new plugin, that config won't go to waste. This is not priority 0 though

### Submitting configuration

Validation is a consideration once we implement plugin options.
There's a list of things we could do for validation. My opinion is that back-end validation is enough. Features:
 - **Backend validation**. Back end validates the options using validators from the manifest. If there are any errors, the errors are displayed in the UI.
 - **Field type validation**. Make sure that field inputs correlate to the manifest, for example number field only accepts numbers, etc.
 - **Regex string**. If there's a regex validator in manifest we could use the same for front-end.

## Island

### Storing the configuration

Validate the config against manifests

## Requirements

Agent needs to know the current config

Island needs to know default and current config

Interfaces need to know current, all available options and descriptions

### Single config vs separate configurations

Single config:
 - Less traffic
 - Would need to merge plugin configurations into it
 - Configurations are getting coupled

Config per plugin:
 - Easier loading
 - Harder resetting and import/export
 - More changes, refactoring repository

### YAML vs JSON vs XML

XML is not readable nor a standard in python. YAML is more readable, but JSON is probably the better option. The plugins will be developed in python most of the time. This means that
it's easier to dump the dictionary you tested with to json (yaml would require additional dependency). Validating YAML with jsonschema is also unorthodox, as to write a plugin you'll have to switch between the two notations. Lastly, YAML to JSONSchema would sacrifice some JSONSchema features, such as refs.

There's also [JDT](https://www.rfc-editor.org/rfc/rfc8927), but it seems to still be in "experimental" stage and it looks like a simplified version of jsonschema.

# Agent

## Running the plugins

Agent needs to run the entrypoint of the plugin and pass the relevant options. Our config should end up pretty much the same, so doesn't matter much


# UI prototype

# Usage

There are no good packages to display YAML as a form (only one I found: https://github.com/MaximeGoyette/react-yaml-form)

`react-jsonschema-form` takes in jsonschema and has everything we need, except UI component versatility.

We should also consider migrating our code to the official `react-jsonschema-form` package. Our current package `react-jsonschema-form-bs4` is 3 years old and unmaintained.

# Validation

`react-jsonschema-form` uses [ajv-validator](https://github.com/ajv-validator/ajv) which is fast and supports everything we need (including [formats](https://react-jsonschema-form.readthedocs.io/en/latest/usage/validation/#customformats)).

By default jsonschema supports these formats (in addition to the simple types):
Dates, Times, and Duration
Email Addresses
Hostnames
IP Addresses
Resource Identifiers
uri-template
JSON Pointers
regex

Corresponding `react-jsonschema-form` formats can be found [here](https://ajv.js.org/guide/formats.html)

Developer can specify the format in schema and if the interface understands it configuration will be validated.
However, if the interface is not familiar with the format, the format can't be defined in the schema.


# GUI

For GUI we need a scalable component to select the plugin (like a dropdown, scrollable menu, etc) and an option window.

You can see some examples in the "gui" folder.

The mock we agreed upon is in "PluginOptionMock.png".
